# bifacial_radiance

In [1]:
import utils
from bokeh.io import output_notebook
from bokeh.plotting import show

C:\Users\KANDERSO\Software\Anaconda3\envs\dev\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\KANDERSO\Software\Anaconda3\envs\dev\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\KANDERSO\Software\Anaconda3\envs\dev\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
output_notebook()

Loading BokehJS ...

## GitHub

In [3]:
gh = utils.get_github_stars('nrel/bifacial_radiance')

In [4]:
p = utils.plot_github_stars_timeseries(gh)
show(p)